In [1]:
import pandas as pd
import stanza
import nltk
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')

2022-03-25 18:08:55 WARNING: Can not find mwt: default from official model list. Ignoring it.
2022-03-25 18:08:55 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

2022-03-25 18:08:55 INFO: Use device: cpu
2022-03-25 18:08:55 INFO: Loading: tokenize
2022-03-25 18:08:55 INFO: Loading: pos
2022-03-25 18:08:56 INFO: Done loading processors!


In [2]:
df = pd.read_excel("datos_features.xlsx")
#df.offense_rating.max()

In [116]:
df.head(5)

,Unnamed: 0,Unnamed: 0.1,id,text,is_humor,humor_rating,humor_controversy,offense_rating,texto_limpio,sentimiento_stanza,...,Religion,Style_of_life,Non_normative,punctuation_symbols_stanza_tagger,primera_plural_stanza,segunda_plural_stanza,tercera_plural_stanza,primera_singular_stanza,segunda_singular_stanza,tercera_singular_stanza
0,0,0,1,TENNESSEE: We're the best state. Nobody even c...,1,2.42,1.0,0.2,tennessee we be the good state nobody even com...,0,...,0,0,0,4,1,0,0,0,0,0
1,1,1,2,A man inserted an advertisement in the classif...,1,2.50,1.0,1.1,a man insert a advertisement in the classified...,1,...,0,0,0,4,0,0,0,0,0,0
2,2,2,3,How many men does it take to open a can of bee...,1,1.95,0.0,2.4,how many man do it take to open a can of beer ...,0,...,0,0,0,2,0,0,0,0,0,1
3,3,3,4,Told my mom I hit 1200 Twitter followers. She ...,1,2.11,1.0,0.0,tell my mom i hit twitter follower she point o...,1,...,0,0,0,2,0,0,0,2,0,0
4,4,4,5,Roses are dead. Love is fake. Weddings are bas...,1,2.78,0.0,0.1,rose be dead love be fake wedding be basically...,1,...,0,0,0,1,0,0,0,0,0,0


### Number distinct of nouns, adjectives, adverbs, pronouns

In [10]:
texto = df['texto_limpio']
postagging = []

for i,row in enumerate(texto):
    words = row.split()
    postagging.append(nltk.pos_tag(words))

noun = []
adv = []
pron = []
adj = []


freq_noun = []
freq_pron = []
freq_advs = []
freq_adjs = []

for i, row in enumerate(postagging):
  #  print('oracion',i)

    nouns = set()
    advs = set()
    prons = set()
    adjs = set()


    for i, tupla in enumerate(row):
        
        if tupla[1]=='NN' or tupla[1]=='NNS' or tupla[1]=='NNP' or tupla[1]=='NNPS':
           # noun.append(tupla[1])
            nouns.add(tupla[0])
           # print(tupla[0])
            
        elif tupla[1]=='PRP' or tupla[1]=='PRP$':
           # pron.append(tupla[1])
            prons.add(tupla[0])
          #  print(tupla[0])
            
        elif tupla[1]=='RB' or tupla[1]=='RBR' or tupla[1]=='RBS':
           # adv.append(tupla[1])
            advs.add(tupla[0])
         #   print(tupla[0])
            
        elif tupla[1]=='JJ' or tupla[1]=='JJR' or tupla[1]=='JJS':
           # adj.append(tupla[1])
            adjs.add(tupla[0])
          #  print(tupla[0])
            
    freq_noun.append(len(nouns))
    freq_pron.append(len(prons))
    freq_advs.append(len(advs))
    freq_adjs.append(len(adjs))

#print(freq_noun,freq_pron, freq_advs, freq_adjs)       
print(len(freq_noun),len(freq_pron), len(freq_advs), len(freq_adjs))       

8000 8000 8000 8000


In [11]:
df['noun_frequency'] = freq_noun
df['pronoun_frequency'] = freq_pron
df['adverbs_frequency'] = freq_advs
df['adjective_frequency'] = freq_adjs

### Punctuation with Stanza

In [33]:
def punctuation(text):
    
    doc = nlp(text)
    puntuacion=set()
    for sent in doc.sentences: 
        for word in sent.words:
            if word.upos == 'PUNCT':
               # print((word.text,word.upos))
                puntuacion.add(word.text)
    return len(puntuacion)

In [36]:
ejemplo=df['text']
lista_puntuacion = []
for i in ejemplo:
    #print(i)
    p = punctuation(i)
    lista_puntuacion.append(p)
len(lista_puntuacion)

8000

In [37]:
df['punctuation_symbols_stanza_tagger'] = lista_puntuacion

### Nº pronouns plural with Stanza

In [108]:
def pronoun_pl(text):
    
    doc = nlp(text)
    conjunto={}
    for word in doc.sentences[0].words: 
        l = set()
        if word.upos == 'PRON':
            if word.feats != None and word.feats.find("|"):
                for caracteristicas in word.feats.split('|'):
                    l.add(caracteristicas)    
                    conjunto[word.text] = l

    # 1ra, 2da y 3ra persona plural

    nprimera_plural=0
    nsegunda_plural=0
    ntercera_plural=0

    for k,v in conjunto.items(): 
        #print(k) para cada pronoun
        for pal in v:
            if pal=='Person=1' and  'Number=Plur' in v:
                nprimera_plural += 1
              #  print('Primera persona plural')

            elif pal=='Person=2' and 'Number=Plur' in v:
                nsegunda_plural += 1
               # print('Segunda persona plural')

            elif pal=='Person=3' and 'Number=Plur' in v:
                ntercera_plural += 1
                #print('Tercera persona plural')
    return nprimera_plural,nsegunda_plural,ntercera_plural

In [109]:
primera_plural=[]
segunda_plural=[]
tercera_plural=[]

for i in df['text']:
    p = pronoun_pl(i)
    primera_plural.append(p[0])
    segunda_plural.append(p[1])
    tercera_plural.append(p[2])
    
len(primera_plural)
#primera_plural,segunda_plural,tercera_plural

8000

In [110]:
df['primera_plural_stanza']=primera_plural
df['segunda_plural_stanza']=segunda_plural
df['tercera_plural_stanza']=tercera_plural

### Nº pronouns singular with Stanza

In [112]:
def pronoun_sg(text):
    
    doc = nlp(text)
    conjunto={}
    for word in doc.sentences[0].words: 
        l = set()
        if word.upos == 'PRON':
            if word.feats != None and word.feats.find("|"):
                for caracteristicas in word.feats.split('|'):
                    l.add(caracteristicas)    
                    conjunto[word.text] = l

    nprimera_singular=0
    nsegunda_singular=0
    ntercera_singular=0

    for k,v in conjunto.items(): 
        #print(k) para cada pronoun
        for pal in v:
            if pal=='Person=1' and  'Number=Sing' in v:
                nprimera_singular += 1
              #  print('Primera persona plural')

            elif pal=='Person=2' and 'Number=Sing' in v:
                nsegunda_singular += 1
               # print('Segunda persona plural')

            elif pal=='Person=3' and 'Number=Sing' in v:
                ntercera_singular += 1
                #print('Tercera persona plural')
    return nprimera_singular,nsegunda_singular,ntercera_singular

In [113]:
primera_singular=[]
segunda_singular=[]
tercera_singular=[]

for i in df['text']:
    p = pronoun_sg(i)
    primera_singular.append(p[0])
    segunda_singular.append(p[1])
    tercera_singular.append(p[2])

In [114]:
df['primera_singular_stanza']=primera_singular
df['segunda_singular_stanza']=segunda_singular
df['tercera_singular_stanza']=tercera_singular

In [115]:
import os
df.to_excel('/Users/luciainesmerlo/Desktop/TFG/HaHackaton2020/datos_features.xlsx')

In [19]:
def singular_person(words,p=1):
    
    if p == 1:
         return any(word in ("i", "me","mine","my", "myself") 
                   for word in words)
    
    elif p == 2:
        return any(word in ("you", "your","yours","yourselfs", "yourself" )
                  for word in words)
    
    else:
        return any(word in ("he", "she","him","his", "himself", "her","hers", "herself","it","its","itself")
                  for word in words)
    
    
def plural_person(words,p=1):
    
    if p == 1:
         return any(word in ("we","us", "our", "ours", "ourselfes")
                   for word in words)
    
    elif p == 2:
        return any(word in ("you", "your","yours", "yourselves" )
                  for word in words)
    
    else:
        return any(word in ("they", "them","thier","theirs", "theirselves")
                  for word in words)

In [25]:
texto = df['texto_limpio']
palabras = []
for i,row in enumerate(texto):
    words = row.split()
    #postagging.append(nltk.pos_tag(words))
    palabras.append(words)
    
first_person_singular = []
second_person_singular = []
third_person_singular = []

first_person_plural = []
second_person_plural = []
third_person_plural = []



for i, pal in enumerate(palabras):
  #  print('oracion',i)
  #  print(pal)
    
    fs_singular = 0
    sd_singular = 0
    td_singular = 0
    
    fs_plural = 0
    sd_plural = 0
    td_plural = 0
    
    if singular_person(pal,1):
        
        fs_singular += 1
        
    elif singular_person(pal,2):
        
        sd_singular += 1
        
    elif singular_person(pal,3):
        
        td_singular += 1
        
    elif plural_person(pal,1):
        
        fs_plural += 1
        
    elif plural_person(pal,2):
        
        sd_plural += 1
        
    elif plural_person(pal,3):
        
        td_plural += 1
    
    first_person_singular.append(fs_singular) 
    second_person_singular.append(sd_singular)
    third_person_singular.append(td_singular)
    
    first_person_plural.append(fs_plural)
    second_person_plural.append(sd_plural)
    third_person_plural.append(td_plural)
    
print(len(first_person_singular),len(second_person_singular),len(third_person_singular),
     len(first_person_plural),len(second_person_plural),len(third_person_plural)) 

df['first_person_singular_pronoun']=first_person_singular
df['second_person_singular_pronoun']=second_person_singular
df['third_person_singular_pronoun']=third_person_singular

df['first_person_plural_pronoun']=first_person_plural
df['second_person_plural_pronoun']=second_person_plural
df['third_person_plural_pronoun']=third_person_plural

8000 8000 8000 8000 8000 8000


In [11]:
import re

def puntuacion():
    
    punt = r"(\(|\)|\?|\¿|\!|\¡|\.\.\.|\,|\'|\"|\.|\;|\:|\%)"
    tres = r"\.{2,5}"
    numeros = r'[0-9][0-9,./%-:]+'
    fecha = r'(\d{2}(/|-)\d{2}((/|-)\d{2,4})?)'
    fecha_pals = r"([0-9]+ de? [\w].+ de? [0-9]+)"
    webs = r'([\w://@]+[\w]+\.[\w|\w-]+\.[\w]+)'
    correo = r'([^@\s]+@[\w]+\.[\w]+\.[\w]+)'
    user = r'(@\w+\-?(\w+)?)'
    hasht = r'(#\w+\-?(\w+)?)'
    acronimos = r'([A-Z]{1,}[.]+(?:[A-z]{1,}[.])*)'
    pals_apos = r"(\b[\w]+'[\w]+)" 
    pals = r"(\b\w+\b)"
    emojis = r"[^\w\s]"


    patrones = '|'.join([tres,fecha_pals,user,pals_apos,pals,hasht,fecha,acronimos,numeros,webs,emojis,correo,punt])
    patron = re.compile(patrones,re.I|re.U)
    return patron

import string
simbolos = [ '(', '"', ')','"' , '(' , ')' ,"|" , "?" ,  "¿" , "!" ,  "¡" , "..." ,
           ",", "'", '.', ';',':','%','#','$','&','}','~','^','_','`','{','[',']','\\',
            '@','*','-','/','<','>','+','\ '] #, '\' 
simbolos = set(simbolos)

texto = df['text']
punct = []
p = puntuacion()
sim = []

for i,row in enumerate(texto):
    simb = set()
   
    for i in p.finditer(row):
        char = row[i.start():i.end()]
        if char in simbolos:
            simb.add(char)
       
    sim.append(len(simb))
    
len(sim)

df['punctuation_symbols'] = sim